In [ ]:
using Interact
using RigidBodyDynamics
using RigidBodyTreeInspector

In [ ]:
# Open the viewer application
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

In [ ]:
# Construct a mechanism
mechanism = rand_chain_mechanism(Float64, [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1:5]]...)

In [ ]:
# Construct the visualizer interface, and get the :robot1 subtree
vis = DrakeVisualizer.Visualizer()[:robot1];

In [ ]:
# Load the mechanism's geometry into the visualizer
setgeometry!(vis, mechanism)

# We can draw the mechanism at a single state:
state = MechanismState{Float64}(mechanism)
settransform!(vis, state);

In [ ]:
# Or we can interactively explore the degrees of freedom of the mechanism.
# Note that the quaternion floating base joint, "joint1", has been given 
# six sliders. The first three correspond to the exponential map representation
# of its rotation, and the next three correspond to its translation. 
inspect(mechanism; show_inertias=true, randomize_colors=true);

In [ ]:
# We can visualize reference frames and points fixed to bodies:
frame = frame_after(last(tree_joints(mechanism)))
point = Point3D(frame, 0.1, 0.2, 0.3)
addgeometry!(vis, mechanism, frame; scale=0.5)
addgeometry!(vis, mechanism, point; radius=0.05)

In [ ]:
# We can also animate the mechanism, given a time sequence of states.
# States will be linearly interpolated between the knot points.
times = collect(linspace(0, 10, 3))
configurations = Vector{Vector{Float64}}(length(times))
for i = 1:length(times)
    rand_configuration!(state)
    configurations[i] = copy(configuration(state))
end
animate(vis, mechanism, times, configurations; fps = 60., realtimerate = 1.)

In [ ]:
delete!(vis)

In [ ]:
# We can simulate a mechanism from an initial state. We'll use a mechanism
# without a QuaternionFloating joint so that it doesn't just fall straight down.
mechanism = rand_chain_mechanism(Float64, [Revolute{Float64} for i = 1:10]...)
delete!(vis)
vis = Visualizer()[:robot2]
setgeometry!(vis, mechanism, Skeleton(false, true))
state = MechanismState{Float64}(mechanism)
settransform!(vis, state)
times, configurations, velocities = simulate(state, 10.; Δt = 0.001);

In [ ]:
# And we can animate the result in realtime
# (note that it looks a little strange because the link center of mass locations aren't where you think they are)
animate(vis, mechanism, times, configurations)

In [ ]:
# We can also inspect individual frames from the simulation
@manipulate for i in 1:length(times)
    set_configuration!(state, configurations[i])
    set_velocity!(state, velocities[i])
    settransform!(vis, state)
    times[i]
end

In [ ]:
# 3D visualization can also be done *while simulating*, using DrakeVisualizerSink in combination with
# the lower level RigidBodyDynamics ODE integration functionality:
using RigidBodyDynamics.OdeIntegrators
result = DynamicsResult{Float64}(mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    damping = 2.
    τ = -damping * velocity(state)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
    nothing
end
integrator = MuntheKaasIntegrator(state, damped_dynamics!, runge_kutta_4(Float64), DrakeVisualizerSink(vis))
integrate(integrator, 10., 1e-3, max_realtime_rate = 1.)